# 4v4m

In [1]:
using Printf, DelimitedFiles
using Plots
using DataFrames
using Revise, MDToolbox

In [2]:
function read_ascii(fname)
    lines = readlines(fname)
    natom = length(lines) - 1
    xyz = zeros(Float64, 1, natom*3)
    for i = 1:natom
        xyz[1, 3*(i-1)+1] = parse(Float64, lines[i+1][31:38])
        xyz[1, 3*(i-1)+2] = parse(Float64, lines[i+1][39:46])
        xyz[1, 3*(i-1)+3] = parse(Float64, lines[i+1][47:54])
    end
    TrjArray(xyz=xyz)
end

read_ascii (generic function with 1 method)

In [3]:
fnames = []
omega = []
phi = []
prou = []
d = []
score = []
rmsd = []
struc = []

Any[]

In [4]:
nrun = 1000

1000

In [5]:
for irun = 1:nrun

    if irun % 100 == 0
        print("irun = $irun")
    end
    for i in 1:100
        filename = "$(irun)/complex.$(i).pdb"
        #t = mdload(filename)
        #t = MDToolbox.readpdb_biostructures(filename)
        t = read_ascii(filename)
        t1 = t[:, 1:1427]
        t2 = t[:, 1428:end]
        #@show size(t1)
        #@show size(t2)
        r = MDToolbox.compute_skrew(t1, t2)
        push!(fnames, filename)
        push!(omega, r.omega)
        push!(phi, r.phi / (2.0 * pi) * 360.0)
        push!(prou, r.prou)
        push!(d, r.d)
        push!(struc, t)
        #t_ca = t["atomname CA"]
        #push!(rmsd, compute_rmsd_permute(ref, t_ca, 2))
    end

    s = readdlm("$(irun)/zdock.out", skipstart=5)
    score = [score; s[1:100, end]]
end

irun = 100irun = 200irun = 300irun = 400irun = 500irun = 600irun = 700irun = 800irun = 900irun = 1000

In [6]:
df = DataFrame(fnames=fnames, score=score, omega=omega, phi=phi, prou=prou, d=d, struc=struc)

Row,fnames,score,omega,phi,prou,d,struc
,Any,Any,Any,Any,Any,Any,Any
1,1/complex.1.pdb,986.974,"[-0.356917, -0.824965, -0.438227]",81.7263,"[88.2692, -72.4848, 64.5618]",-31.825,"1x2854 TrjArray{Float64, Int64}\n| 92.96 -13.72 82.61 | … 112.08 18.12 98.36 |"
2,1/complex.2.pdb,982.884,"[0.0686623, -0.995647, 0.0630238]",177.85,"[113.11, 14.442, 104.925]",-0.692541,"1x2854 TrjArray{Float64, Int64}\n| 92.96 -13.72 82.61 | … 131.95 -14.84 108.30 |"
3,1/complex.3.pdb,962.178,"[-0.328547, 0.882951, -0.335343]",129.578,"[68.673, 48.0899, 59.3385]",-29.4352,"1x2854 TrjArray{Float64, Int64}\n| 92.96 -13.72 82.61 | … 118.05 -52.66 82.45 |"
4,1/complex.4.pdb,940.885,"[-0.322728, -0.924926, 0.200894]",84.9772,"[99.9138, -16.8225, 83.0557]",-16.4955,"1x2854 TrjArray{Float64, Int64}\n| 92.96 -13.72 82.61 | … 83.55 0.05 81.92 |"
5,1/complex.5.pdb,937.892,"[-0.052929, 0.218289, 0.974448]",99.5387,"[117.498, -69.8142, 22.0214]",-16.2941,"1x2854 TrjArray{Float64, Int64}\n| 92.96 -13.72 82.61 | … 83.85 -72.13 99.42 |"
6,1/complex.6.pdb,919.653,"[0.288118, 0.0219858, 0.957343]",75.2528,"[81.7794, -53.1156, -23.3921]",-19.8376,"1x2854 TrjArray{Float64, Int64}\n| 92.96 -13.72 82.61 | … 78.19 -62.68 91.28 |"
7,1/complex.7.pdb,919.264,"[-0.373651, 0.798232, -0.472451]",144.723,"[59.4401, 51.316, 39.6914]",-29.008,"1x2854 TrjArray{Float64, Int64}\n| 92.96 -13.72 82.61 | … 117.81 -57.96 83.62 |"
8,1/complex.8.pdb,914.768,"[0.312959, 0.256981, 0.91434]",76.8921,"[74.4597, -61.2717, -8.26513]",22.47,"1x2854 TrjArray{Float64, Int64}\n| 92.96 -13.72 82.61 | … 106.33 -34.42 131.36 |"
9,1/complex.9.pdb,911.393,"[-0.869092, -0.48917, -0.0734266]",154.197,"[41.397, -88.9174, 102.386]",5.9597,"1x2854 TrjArray{Float64, Int64}\n| 92.96 -13.72 82.61 | … 126.11 -77.93 96.92 |"


In [7]:
df_org = deepcopy(df);

In [8]:
#using BSON: @save, @load
#@save "skrew_parameters.bson" fnames score omega phi prou d df rmsd

In [9]:
#using BSON: @save, @load
#@save "df.bson" df

In [10]:
using JLD2

save("skrew_parameters.jld2", "fnames", fnames, "score", score, "omega", omega, "phi", phi, "prou", prou, "d", d, "df", df, "rmsd", rmsd)
save("df.jld2", "df", df)